# SQL Project

Analysis for startup project for book lovers.  The goal is to generate a value proposition for a new product.

#### Task
- Find the number of books released after January 1, 2000.
- Find the number of user reviews and the average rating for each book.
- Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis).
- Identify the author with the highest average book rating: look only at books with at least 50 ratings.
- Find the average number of text reviews among users who rated more than 50 books.


## Content plan

1. [Step1](#Step1) Studying the tables 
2. [Step2](#Step2) Making an SQL query for each of the tasks
3. [Step3](#Step3) Conclusions


### Step1

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'practicum_student',         # username
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-final-project-db'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
books = '''SELECT
           *
           FROM
           books
           LIMIT 5'''
pd.io.sql.read_sql(books, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
authors = '''SELECT
           *
           FROM
           authors
           LIMIT 5'''
pd.io.sql.read_sql(authors, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
reviews = '''SELECT
             *
             FROM
             reviews
             LIMIT 5'''
pd.io.sql.read_sql(reviews, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [5]:
ratings = '''SELECT
             *
             FROM
             ratings
             LIMIT 5'''
pd.io.sql.read_sql(ratings, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
publishers = '''SELECT
             *
             FROM
             publishers
             LIMIT 5'''
pd.io.sql.read_sql(publishers, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### Step2

**Find the number of books released after January 1, 2000**

In [7]:
n_books = '''SELECT 
             COUNT (DISTINCT book_id) AS number_of_books
             FROM books
             WHERE publication_date >= '2000-01-01'
             '''

pd.io.sql.read_sql(n_books, con = engine)

,number_of_books
0,821


There are 821 books released after 01-01-2000

**Find the number of user reviews and the average rating for each book**

In [13]:
rrbooks='''SELECT 
           books.book_id AS id,
           books.title AS title,
           COUNT(DISTINCT reviews.review_id) AS cnt_review,
           AVG(ratings.rating) AS avg_rating
           FROM books
           LEFT JOIN ratings ON books.book_id=ratings.book_id
           LEFT JOIN reviews ON books.book_id=reviews.book_id
           GROUP BY id, title
           ORDER BY
           cnt_review DESC,
           avg_rating DESC
           LIMIT 5'''

pd.io.sql.read_sql(rrbooks, con = engine)

,id,title,cnt_review,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,656,The Book Thief,6,4.264151
4,734,The Glass Castle,6,4.206897


The leader in the number of reviews is Twilight, the highest rating has Harry Potter and the Prisoner of Azkaban

**Identify the publisher that has released the greatest number of books with more than 50 pages (this will help you exclude brochures and similar publications from your analysis)**

In [9]:
publisher='''SELECT 
             publishers.publisher AS publisher,
             COUNT(DISTINCT books.book_id) AS cnt_books
             FROM books
             LEFT JOIN publishers ON books.publisher_id=publishers.publisher_id
             WHERE books.num_pages > 50
             GROUP BY publisher
             ORDER BY
             cnt_books DESC
             LIMIT 5'''

pd.io.sql.read_sql(publisher, con = engine)

,publisher,cnt_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19


Top Publishers in terms of release the greatest number of books are Penguin Books, Vintage, Grand Central Publishing.

**Identify the author with the highest average book rating: look only at books with at least 50 ratings**

In [10]:
author='''SELECT
          authors.author AS author,
          AVG(ratings.rating) AS avg_rating,
          COUNT(DISTINCT ratings.rating_id) AS cnt_rating
          FROM books
          LEFT JOIN authors ON books.author_id = authors.author_id
          LEFT JOIN ratings ON books.book_id = ratings.book_id
          GROUP BY author
          HAVING COUNT(DISTINCT ratings.rating_id) >= 50
          ORDER BY avg_rating DESC
          limit 3'''

pd.io.sql.read_sql(author, con = engine)

,author,avg_rating,cnt_rating
0,Diana Gabaldon,4.300000,50
1,J.K. Rowling/Mary GrandPré,4.288462,312
2,Agatha Christie,4.283019,53


The author with the highest average book rating is Diana Gabaldon

**Find the average number of text reviews among users who rated more than 50 books**

In [11]:
review='''SELECT
          AVG(Sub.cnt_text) AS avg_cnt_text
          FROM
              (SELECT ratings.username,
               COUNT(DISTINCT reviews.text) AS cnt_text,
               COUNT(DISTINCT ratings.book_id) AS cnt_books
               FROM ratings
               JOIN reviews ON ratings.username = reviews.username
               GROUP BY ratings.username
               HAVING COUNT(DISTINCT ratings.book_id) > 50) AS Sub
          '''

pd.io.sql.read_sql(review, con = engine)

,avg_cnt_text
0,24.333333


The average number of text reviews is 24.3

### Step3

Conclusions:
- There are 821 books released after 01-01-2000
- The leader in the number of reviews is Twilight, the highest rating has Harry Potter and the Prisoner of Azkaban
- Top Publisher in terms of release the greatest number of books is Penguin Books
- The author with the highest average book rating is Diana Gabaldon
- The average number of text reviews is 24.3